In [1]:
infile = "/data/Data/yntn/data_cleaned_10M.log"
outfile = "/data/Data/yntn/data_out.csv"
emoji_path = '/data/Data/feed_data/new_cat'

In [2]:
with open(infile, "r") as f:
    lines = f.readlines()   

In [3]:
import json

# read file
with open('emoji2code.json', 'r') as myfile:
    data = myfile.read()

# parse file
obj = json.loads(data)
emoji = obj.keys()

In [4]:
from pathlib import Path

emoji_dict = {}

for file_dir in Path(emoji_path).glob('**/*.png'):
    if (str(file_dir.parent.stem) in emoji_dict):  
        emoji_dict[str(file_dir.parent.stem)].append(str(file_dir.stem))
    else:
        emoji_dict[str(file_dir.parent.stem)] = [str(file_dir.stem)]

In [6]:
emoji_dict

{'Cười lớn ': ['emoji_10'],
 'Vỗ tay ': ['emoji_671'],
 'Buồn': ['emoji_29', 'emoji_109', 'emoji_24', 'emoji_34'],
 'Suy ngẫm': ['emoji_8', 'emoji_95'],
 'Ngủ ngon': ['emoji_32', 'emoji_107'],
 'MỆT': ['emoji_116', 'emoji_37', 'emoji_47'],
 'Lêu lêu ': ['emoji_16', 'emoji_98'],
 'CẢM ƠN': ['emoji_78', 'emoji_139'],
 'Hôn': ['emoji_12', 'emoji_2'],
 'Ngầu': ['emoji_57', 'emoji_21'],
 'Mắng': ['emoji_53', 'emoji_121'],
 'Sợ': ['emoji_1023', 'emoji_1032'],
 'TIỀN': ['emoji_62'],
 'Đồng ý ': ['emoji_92', 'emoji_80', 'emoji_4', 'emoji_141'],
 'Yêu': ['emoji_94',
  'emoji_144',
  'emoji_3',
  'emoji_91',
  'emoji_84',
  'emoji_7'],
 'THẮC MẮC': ['emoji_117', 'emoji_26'],
 'Khóc lớn': ['emoji_19'],
 'CỐ LÊN': ['emoji_122', 'emoji_88', 'emoji_79', 'emoji_140', 'emoji_56'],
 'BẠO LỰC': ['emoji_104',
  'emoji_54',
  'emoji_131',
  'emoji_69',
  'emoji_50',
  'emoji_70',
  'emoji_28'],
 'Giận': ['emoji_99',
  'emoji_18',
  'emoji_108',
  'emoji_49',
  '

In [5]:
import operator
import csv
import re, string
 
with open(outfile, "w") as f:
    writer = csv.writer(f)
    myData = [['text','label']]
    for line in lines:
        if ("feed at" in line):
            continue        
            
        if ('ttt' in text):
            continue
               
        count = {}
        st = line
        for phrase in emoji:
            if (phrase in st):
                count[obj[phrase]] = 0
                while (phrase in st):
                    count[obj[phrase]] = count[obj[phrase]] + 1
                    st = st.replace(phrase,'',1)                    
        emoji_max = max(count.items(), key=operator.itemgetter(1))[0] #string
        
        st = st.lower()
        words = st.split()
        words = re.split(r'\W+', st)
        text = ''        
        text = ' '.join(word for word in words)
        text = text.strip()
             
        if (text == "") or (type(text) == float) or (str(text) =='nan'): 
            continue         
        for etype in emoji_dict.keys():
            if emoji_max in emoji_dict[etype]:
                myData.append([text, etype.lower()])
                break
                
    writer.writerows(myData)

In [6]:
import pandas as pd 
data = pd.read_csv('/data/Data/yntn/data_out.csv') 
data

,text,label
0,ngon k c,suy ngẫm
1,sao e la i no i nv bô c hung dư lă m ha,khóc lớn
2,ok n cho cái hẹn đi,vui
3,thùy mị quá bạn ah,yêu
4,có đâu xé ib đi,ngạc nhiên
5,sao cũg đc hết,vui
6,e muốn ở tn mà,khóc lớn
7,e cũng ngủ ngon nhé,hôn
8,a ở lại đi,vui
9,lâu không gặp nhìn đẹp zai hẳn nha,cười gian
